In [1]:
import schedule
import time
from gtts import gTTS
import os
from playsound import playsound
playsound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/startup.mp3')

In [2]:
playsound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/MARVIN1.mp3')
import winsound
import numpy as np
import pandas as pd
import math
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sympy import *
init_printing()
import xlrd
from datetime import datetime as dt
#import socketio

In [3]:
#import requests
import json
from scipy.signal import argrelextrema
import pytz
import itertools
from itertools import combinations
from itertools import product
from itertools import groupby
from astropy.table import QTable, Table, Column
from astropy import units as u
from IPython.display import HTML
#import fxcmpy

In [4]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5

In [5]:
if not mt5.initialize(path="C:/BillyL/MetaTrader 5/terminal64.exe",login=1046003, server="TradersWay-Demo",password="j96RQnhb"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [6]:
##  INITIAL SETUP



## WATCHLIST = ['AUD/CAD', 'AUD/CHF', 'AUD/JPY', 'AUD/NZD', 'AUD/USD', 'CAD/CHF', 'CAD/JPY', 'CHF/JPY', 'EUR/AUD', 'EUR/CAD', 'EUR/CHF', 'EUR/GBP', 'EUR/JPY', 'EUR/NZD', 'EUR/USD', 'GBP/AUD', 'GBP/CAD', 'GBP/CHF', 'GBP/JPY', 'GBP/NZD', 'GBP/USD', 'NZD/CAD', 'NZD/CHF', 'NZD/JPY', 'NZD/USD', 'USD/CAD', 'USD/CHF', 'USD/JPY', 'XAG/USD', 'XAU/USD', 'BCH/USD', 'BTC/USD', 'EOS/USD', 'ETH/USD', 'LTC/USD', 'XLM/USD', 'XRP/USD']

## FOREX = ## WATCHLIST = ['AUDCAD','AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY','EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD','GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPUSD','NZDCAD','NZDCHF','NZDJPY','NZDUSD','USDCAD','USDCHF','USDJPY']

#TOKEN = 'b199bb5a15fa26957990a3fb823a51bd473cca55'
#con = fxcmpy.fxcmpy(access_token=TOKEN, log_level='error', server='demo', log_file='log.txt')


pair = ['AUDCAD','AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY','EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD','GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPUSD','NZDCAD','NZDCHF','NZDJPY','NZDUSD','USDCAD','USDCHF','USDJPY']

def brutus():

    
    h = len(pair)

    #column_names = ["pairing","position","entry","tkprofit","stloss"]
    #purps = pd.DataFrame(columns = column_names)
    purps = []

    for x in pair:

        Srates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_H8, 0, 600)

        S = pd.DataFrame(Srates)
        S['time']=pd.to_datetime(S['time'], unit='s')
        S = S.drop(columns=['tick_volume', 'spread', 'real_volume'])
        S = S.set_index('time')
        S = S.tz_localize('Etc/GMT-7')
        S = S.tz_convert('UTC')

        S = S.assign(MOVE=(S['close'] - S['open']))
        SAvgCandle = np.average(abs(S['MOVE']))
        STDCandle = np.std(abs(S['MOVE']))
        CP = S['close'].iloc[-1]
        scope = SAvgCandle*15
        SCM = SAvgCandle

        CNDL = (S.iloc[:, 3]) - (S.iloc[:, 0])
        S['CNDL'] = np.where(CNDL > 0, 1, -1)
        S = S.assign(INFL=S['CNDL'].ne(S['CNDL'].shift()))
        S = S[S.INFL != 0]
        S['INFL'] = S['CNDL']*-1
        S['LVL'] = np.where(S.INFL == 1, S.high, S.low)
        S = S.assign(SIZE=abs(S['LVL'] - (S['LVL'].shift(-1))))

        A = np.array(S.LVL)

        B = list(combinations(A, 2))
        Ba = np.array([x for x, y in B])
        B = np.array([abs(x - y) for x, y in B])
        Bb = np.array([np.where(x <= SCM, 1, 0) for x in B])

        T = list(zip(Ba,Bb))

        table = pd.DataFrame(T, columns=['a','b']).groupby('a').sum()
        table['max'] = table.b[(table.b.shift(1) < table.b) & (table.b.shift(-1) < table.b)]
        table = table.sort_values(['a'], ascending=False)
        table = table.rename(columns={'b': 'Pings'})
        table = table.fillna(0)
        table = table[table['max'] != 0]
        pingsavg = np.average(table['Pings'])
        table = table.assign(level=table.index)

        levels = list(table['level'])




        Drates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_M5, 0, 600)

        D = pd.DataFrame(Drates)
        D['time']=pd.to_datetime(D['time'], unit='s')
        D = D.drop(columns=['tick_volume', 'spread', 'real_volume'])
        D = D.set_index('time')
        D = D.tz_localize('Etc/GMT-7')
        D = D.tz_convert('UTC')

        D = D.assign(MOVE=(D['close'] - D['open']))
        DAvgCandle = np.average(abs(D['MOVE']))
        skew = 1
        CM = DAvgCandle*skew
        l = int(D['close'].count())
        lin = np.linspace(1,l,num=l)

        D = D.assign(STICK=(D['high'] - D['low']))
        D = D.assign(PSTICK=abs(D['MOVE']/D['STICK']))
        D['DIRECT'] = np.where(D.MOVE > 0, 'Bullish', 'Bearish')

        ANIMAL = []
        i = 0
        for t in lin: 
            if D.iloc[i,6] <= 1.1:
                if D.iloc[i,6] < 0.85:
                    if D.iloc[i,6] < 0.65:
                        if D.iloc[i,6] < 0.50:
                            if D.iloc[i,6] <= 0.30:
                                if D.iloc[i,6] <= 0.15:
                                    animal = 'Doji'
                                else:
                                    animal = 'Frog'
                            else:
                                animal = 'Cat'
                        else:
                            animal = 'Dog'
                    else:
                        animal = 'Horse'
                else:
                    animal = 'Shark'
            ANIMAL.append(animal)
            i = i + 1
        D['ANIMAL'] = ANIMAL

        GAUGE = []
        i = 0
        for t in lin: 
            cndmid = (D.iloc[i,0]+D.iloc[i,3])/2
            stickmid = (D.iloc[i,1]+D.iloc[i,2])/2
            if D.iloc[i,5] == 0:
                midpoint = D.iloc[i,0]
            else:
                midpoint = (cndmid - stickmid)/D.iloc[i,5]
            if midpoint >= 0.075:
                if midpoint >= 0.175:
                    if midpoint >= 0.25:
                        gauge = 'Top Saturated'
                    else:
                        gauge = 'Top Stacked'
                else:
                    gauge = 'Top Semi-Stacked'
            elif midpoint <= -0.075:
                if midpoint <= -0.175:
                    if midpoint <= -0.25:
                        gauge = 'Bottom Saturated'
                    else:
                        gauge = 'Bottom Stacked'
                else:
                    gauge = 'Bottom Semi-Stacked'
            else:
                gauge = 'Centered'
            GAUGE.append(gauge)
            i = i + 1
        D['GAUGE'] = GAUGE

        thh = 168
        tl = l - thh
        tlin = np.linspace(1,tl,num=tl)
        tlinb = np.linspace(1,thh,num=thh)
        TREND = []
        for t in tlinb:
            TREND.append(0)
        i = thh                                                                                       
        for t in tlin:
            ii = i - thh
            movetrend = np.sum(list(D.iloc[ii:i,4]))
            i = i + 1
            TREND.append(movetrend)
        D['TREND'] = TREND

        READING=[]
        i = 0
        for t in lin: 
            if D.iloc[i,10] < 0:
                if D.iloc[i,9] == 'Top Saturated':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'JACK'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'DIPPER'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'HAMMER'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'DRAGONFLY DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Top Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'JACK'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'SKILLET'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'MALLET'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'DRAGONFLY DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Top Semi-Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Centered':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Bottom Semi-Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Bottom Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'BOOST'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'BRAKE'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'BRAKE'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Bottom Saturated':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'BOOST'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'BRAKE'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'BRAKE'
                    else:
                        Read = D.iloc[i,8]
                else:
                    Read = D.iloc[i,8]
            elif D.iloc[i,10] > 0:
                if D.iloc[i,9] == 'Bottom Saturated':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'JACK'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'DIPPER'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'SHOOTING STAR'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'GRAVESTONE DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Bottom Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'JACK'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'SKILLET'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'ASTEROID'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'GRAVESTONE DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Bottom Semi-Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Centered':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Top Semi-Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Top Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'BOOST'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'BRAKE'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'BRAKE'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Top Saturated':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'BOOST'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'BRAKE'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'BRAKE'
                    else:
                        Read = D.iloc[i,8]
                else:
                    Read = D.iloc[i,8]
            else:
                Read = D.iloc[i,8]
            i = i + 1
            READING.append(Read)
        D['READING'] = READING

        READING2=[]
        READING2.append(0)
        jl = l - 1
        linjj = np.linspace(1,jl,num=jl)
        i = 1
        for t in linjj: 
            ii = i - 1
            if D.iloc[i,10] < 0:
                if D.iloc[ii,7] == 'Bearish':
                    if D.iloc[i,3] > D.iloc[ii,1]:
                        if D.iloc[ii,11] == 'BRAKE':
                            if D.iloc[i,7] == 'Bullish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BULLISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BULLISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'GLIDE':
                            if D.iloc[i,7] == 'Bullish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BULLISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BULLISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'PLUMP DOJI':
                            if D.iloc[i,7] == 'Bullish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BULLISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BULLISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'LONG-LEG DOJI':
                            if D.iloc[i,7] == 'Bullish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BULLISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BULLISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    elif D.iloc[i,0] == D.iloc[ii,3]:
                        if D.iloc[i,2] == D.iloc[ii,2]:
                            if D.iloc[i,7] == 'Bullish':
                                if D.iloc[i,3] > D.iloc[ii,0]:
                                    Read2 = 'TWEEZERS BOTTOM'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    elif D.iloc[i,3] > (D.iloc[ii,0]+D.iloc[ii,3])/2:
                        if D.iloc[i,0] < D.iloc[ii,3]:
                            if D.iloc[ii,7] == 'Bullish':
                                Read2 = 'PIERCING PATTERN'
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    else:
                        Read2 = 0
                else:
                    Read2 = 0
            elif D.iloc[i,10] > 0:
                if D.iloc[ii,7] == 'Bullish':
                    if D.iloc[i,3] < D.iloc[ii,2]:
                        if D.iloc[ii,11] == 'BRAKE':
                            if D.iloc[i,7] == 'Bearish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BEARISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BEARISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'GLIDE':
                            if D.iloc[i,7] == 'Bearish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BEARISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BEARISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'PLUMP DOJI':
                            if D.iloc[i,7] == 'Bearish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BEARISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BEARISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'LONG-LEG DOJI':
                            if D.iloc[i,7] == 'Bearish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BEARISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BEARISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    else:
                        Read2 = 0
                elif D.iloc[i,3] == D.iloc[ii,0]:
                    if D.iloc[i,1] == D.iloc[ii,1]:
                        if D.iloc[i,7] == 'Bearish':
                            if D.iloc[i,3] < D.iloc[ii,0]:
                                Read2 = 'TWEEZERS TOP'
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    else:
                        Read2 = 0
                elif D.iloc[i,3] < (D.iloc[ii,0]+D.iloc[ii,3])/2:
                    if D.iloc[i,0] > D.iloc[ii,3]:
                        if D.iloc[ii,7] == 'Bearish':
                            Read2 = 'DARK CLOUD COVER'
                        else:
                            Read2 = 0
                    else:
                        Read2 = 0
                else:
                    Read2 = 0
            else:
                Read2 = 0
            i = i + 1
            READING2.append(Read2)
        D['READING2'] = READING2

        OTESTLVL = []
        i = 0
        for t in lin:
            levl = 0
            for u in levels:
                if abs(D.iloc[i,0]-u) <= CM:
                    levl = u
            OTESTLVL.append(levl)
            i = i + 1
        D['OTESTLVL'] = OTESTLVL

        CTESTLVL = []
        i = 0
        for t in lin:
            levl = 0
            for u in levels:
                if abs(D.iloc[i,3]-u) <= CM:
                    levl = u
            CTESTLVL.append(levl)
            i = i + 1
        D['CTESTLVL'] = CTESTLVL

        BREAKLVL = []
        i = 0
        for t in lin:
            levl = 0
            for u in levels:
                if D.iloc[i,7] == 'Bullish':
                    if D.iloc[i,3] > (u+CM):
                        if D.iloc[i,0] < u:
                            levl = u
                        elif D.iloc[i,13] == u:
                            levl = u
                elif D.iloc[i,7] == 'Bearish':
                    if D.iloc[i,3] < (u-CM):
                        if D.iloc[i,0] > u:
                            levl = u
                        elif D.iloc[i,13] == u:
                            levl = u
            BREAKLVL.append(levl)
            i = i + 1
        D['BREAKLVL'] = BREAKLVL

        
        Frates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_M15, 0, 200)

        F = pd.DataFrame(Frates)
        F['time']=pd.to_datetime(F['time'], unit='s')
        F = F.drop(columns=['tick_volume', 'spread', 'real_volume'])
        F = F.set_index('time')
        F = F.tz_localize('Etc/GMT-7')
        F = F.tz_convert('UTC')
        fl = F['close'].count()
        x1 = np.linspace(1, fl, num=fl)
        y1 = F.iloc[:,3]
        m = sm.OLS(y1, x1).fit()
        slope, intercept, r_value, p_value, std_err = stats.linregress(x1,y1)
        wholeslope = slope

        DITREND = []
        i = 0
        for t in lin: 
            if wholeslope > 0:
                if D.iloc[i,10] > 0:
                    ditrend = 1
                elif D.iloc[i,10] < 0:
                    ditrend = -1
                else:
                    ditrend = 0
            elif wholeslope < 0:
                if D.iloc[i,10] < 0:
                    ditrend = 1
                elif D.iloc[i,10] > 0:
                    ditrend = -1
                else:
                    ditrend = 0
            else:
                ditrend = 0
            DITREND.append(ditrend)                
            i = i + 1
        D['DITREND'] = DITREND

        DSTREND = []
        i = 0
        for t in lin: 
            if D.iloc[i,7] == 'Bullish':
                if D.iloc[i,10] > 0:
                    dstrend = 1
                elif D.iloc[i,10] < 0:
                    dstrend = -1
                else:
                    dstrend = 0
            elif D.iloc[i,7] == 'Bearish':
                if D.iloc[i,10] < 0:
                    dstrend = 1
                elif D.iloc[i,10] > 0:
                    dstrend = -1
                else:
                    dstrend = 0
            else:
                dstrend = 0
            DSTREND.append(dstrend)                
            i = i + 1
        D['DSTREND'] = DSTREND

        ypa = ['JACK', 'GLIDE']
        grpa = ['MARIBOZU', 'TUBE', 'BOOST']
        goldpa = ['HAMMER', 'SHOOTING STAR', 'DRAGONFLY DOJI', 'GRAVESTONE DOJI', 'LING-LEG DOJI']
        silvpa = ['PLUMP DOJI', 'MALLET', 'ASTEROID', 'DIPPER', 'SKILLET']
        goldpa2 = ['BULLISH ENGULFING', 'BEARISH ENGULFING', 'TWEEZERS TOP', 'TWEEZERS BOTTOM', 'DARK CLOUD COVER', 'PIERCING PATTERN']
        COLOR = []
        i = 0
        for t in lin:
            color = 'white'
            if D.iloc[i,15] != 0:
                if D.iloc[i,14] == 0:
                    if D.iloc[i,17] == 1:
                        color = 'pearl'
                    elif D.iloc[i,17] == -1:
                        color = 'black'
            elif D.iloc[i,15] == 0:
                if D.iloc[i,14] != 0:
                    if D.iloc[i,13] == D.iloc[i,14]:
                        if D.iloc[i,12] == 0:
                            color = 'gray'
                    elif D.iloc[i,13] == 0:
                        if D.iloc[i,12] == 0:
                            if D.iloc[i,17] == 1:
                                color = 'blue'
                            elif D.iloc[i,17] == -1:
                                color = 'purple'
                        elif D.iloc[i,12] != 0:
                            for ugo2 in goldpa2:
                                if D.iloc[i,12] == ugo2:
                                    color = 'gold'
                                elif D.iloc[i,12] == 0:
                                    if D.iloc[i,11] != 0:
                                        for ugo in goldpa:
                                            if D.iloc[i,11] == ugo:
                                                color = 'gold'
                    elif D.iloc[i,13] == 0:
                        if D.iloc[i,12] == 0:
                            for uy in ypa:
                                if D.iloc[i,11] == uy:
                                    if D.iloc[i,17] == 1:
                                        color = 'yellow'
                                    elif D.iloc[i,17] == -1:
                                        color = 'orange'
                            for ug in grpa:
                                if D.iloc[i,11] == ug:
                                    if D.iloc[i,17] == 1:
                                        color = 'green'
                                    elif D.iloc[i,17] == -1:
                                        color = 'red'
            COLOR.append(color)
            i = i + 1
        D['COLOR'] = COLOR


        Hrates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_H2, 0, 25)

        H = pd.DataFrame(Hrates)
        H['time']=pd.to_datetime(H['time'], unit='s')
        H = H.drop(columns=['tick_volume', 'spread', 'real_volume'])
        H = H.set_index('time')
        H = H.tz_localize('Etc/GMT-7')
        H = H.tz_convert('UTC')
        H = H.assign(MOVE=(H['close'] - H['open']))
        hl = int(H['close'].count())
        hlin = np.linspace(1,hl,num=hl)

        SLOPE = []
        i = 0
        for t in hlin:
            ii = i - 1
            if i == 0:
                SLOPE.append(0)
            else:
                slope = H.iloc[i,3] - H.iloc[ii,3]
                SLOPE.append(slope)
            i = i + 1

        SLOPE2 = []
        i = 0
        for t in hlin:
            ii = i - 1
            if i == 0:
                SLOPE2.append(0)
            else:
                slope2 = SLOPE[i] - SLOPE[ii]
                SLOPE2.append(slope2)
            i = i + 1

        CONCAVE = []
        i = 0
        for t in hlin:
            if SLOPE2[i] == 0:
                CONCAVE.append('INFL')
            elif SLOPE2[i] > 0:
                CONCAVE.append('UP')
            else:
                CONCAVE.append('DOWN')
            i = i + 1

        axiom = D['close'].count()/H['close'].count()
        AXCONCAVE = []
        i = 1
        for t in lin:
            alpha = int(math.ceil(i/axiom)-1)
            axconcave = CONCAVE[alpha]
            AXCONCAVE.append(axconcave)
            i = i + 1
        D['CONCAVE'] = AXCONCAVE

        COLTRADE = []
        i = 0
        for t in lin:
            if i == 0:
                trade = 0
            else:
                if D.iloc[i,18] == 'purple':
                    trade = 'TRADE'
                elif D.iloc[i,18] == 'gold':
                    trade = 'TRADE'
                else:
                    trade = 0
            COLTRADE.append(trade)
            i = i + 1
        D['COLTRADE'] = COLTRADE

        CONTRADE = []
        i = 0
        for t in lin:
            if D.iloc[i,20] == 'TRADE':
                if D.iloc[i,10] > 0:
                    if D.iloc[i,19] == 'UP':
                        contrade = 'TRADE'
                elif D.iloc[i,10] < 0:
                    if D.iloc[i,19] == 'DOWN':
                        contrade = 'TRADE'
            else:
                contrade = 0
            CONTRADE.append(contrade)
            i = i + 1
        D['CONTRADE'] = CONTRADE

        POSITION = []
        i = 0
        for t in lin:
            if D.iloc[i,20] == 'TRADE':
                if D.iloc[i,21] == 'TRADE':
                    if D.iloc[i,10] > 0:
                        position = 'LONG'
                    elif D.iloc[i,10] < 0:
                        position = 'SHORT'
            else:
                position = 0
            POSITION.append(position)
            i = i + 1
        D['POSITION'] = POSITION

        ENTRY = []
        i = 0
        for t in lin:
            if D.iloc[i,22] == 'LONG':
                entry = D.iloc[i,14] + CM
            elif D.iloc[i,22] == 'SHORT':
                entry = D.iloc[i,14] - CM
            else:
                entry = 0
            i = i + 1
            ENTRY.append(entry)
        D['ENTRY'] = ENTRY

        TAKEPROFIT = []
        STOPLOSS = []
        cend = int(len(levels)-1)
        i = 0
        for t in lin:
            if D.iloc[i,22] == 'LONG':
                clevel = D.iloc[i,14]
                clindex = levels.index(clevel)
                tpindex = int(clindex - 1)
                slindex = int(clindex + 1)
                if clindex == 0:
                    stoploss = levels[slindex]
                    sldist = clevel - stoploss
                    takeprofit = clevel + sldist
                elif clindex == cend:
                    takeprofit = levels[tpindex]
                    tpdist = takeprofit - clevel
                    stoploss = clevel - tpdist
                else:
                    takeprofit = levels[tpindex]
                    stoploss = levels[slindex]
            elif D.iloc[i,22] == 'SHORT':
                clevel = D.iloc[i,14]
                clindex = levels.index(clevel)
                tpindex = int(clindex + 1)
                slindex = int(clindex - 1)
                if clindex == 0:
                    takeprofit = levels[tpindex]
                    tpdist = clevel - takeprofit
                    stoploss = clevel + tpdist
                elif clindex == cend:
                    stoploss = levels[slindex]
                    sldist = stoploss - clevel
                    takeprofit = clevel - sldist
                else:
                    takeprofit = levels[tpindex]
                    stoploss = levels[slindex]
            else:
                takeprofit = 0
                stoploss = 0
            i = i + 1
            TAKEPROFIT.append(takeprofit)
            STOPLOSS.append(stoploss)
        D['TAKEPROFIT'] = TAKEPROFIT
        D['STOPLOSS'] = STOPLOSS

        shrink = 0.90
        i = 0
        for t in lin:
            if D.iloc[i,22] == 'LONG':
                entry = D.iloc[i,23]
                tpdist = D.iloc[i,24]-D.iloc[i,23]
                takeprofit = entry + tpdist*shrink
                D.iloc[i,24] = takeprofit
            elif D.iloc[i,22] == 'SHORT':
                entry = D.iloc[i,23]
                tpdist = D.iloc[i,23]-D.iloc[i,24]
                takeprofit = entry - tpdist*shrink
                D.iloc[i,24] = takeprofit
            i = i + 1

        i = 0
        for t in lin:
            if D.iloc[i,22] == 'LONG':
                entry = D.iloc[i,23]
                tpdist = D.iloc[i,24]-D.iloc[i,23]
                sldist = D.iloc[i,23]-D.iloc[i,25]
                if sldist > tpdist:
                    sldist = tpdist
                    stoploss = entry-sldist
                    D.iloc[i,25] = stoploss
            elif D.iloc[i,22] == 'SHORT':
                entry = D.iloc[i,23]
                tpdist = D.iloc[i,23]-D.iloc[i,24]
                sldist = D.iloc[i,25]-D.iloc[i,23]
                if sldist > tpdist:
                    sldist = tpdist
                    stoploss = entry+sldist
                    D.iloc[i,25] = stoploss
            i = i + 1

        D.drop(D.tail(1).index,inplace=True)
        ldc = int(D['close'].count())
        la = int(ldc - 1)
        if D.iloc[la,22] != 0:
            pairing = x
            colcode = D.iloc[la,18]
            position = D.iloc[la,22]
            entry = D.iloc[la,23]
            tkprofit = D.iloc[la,24]
            stloss = D.iloc[la,25]
            df = (pairing,position,entry,tkprofit,stloss)
            purps.append(df)
            
            symbol = pairing
            symbol_info = mt5.symbol_info(symbol)
            if symbol_info is None:
                print(symbol, "not found, can not call order_check()")
                mt5.shutdown()
                quit()

            # if the symbol is unavailable in MarketWatch, add it
            if not symbol_info.visible:
                print(symbol, "is not visible, trying to switch on")
                if not mt5.symbol_select(symbol,True):
                    print("symbol_select({}}) failed, exit",symbol)
                    mt5.shutdown()
                    quit()
                    
            lot = 0.5
            point = mt5.symbol_info(symbol).point
            price = mt5.symbol_info_tick(symbol).ask
            deviation = 20

            if position == 'LONG':
                request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": lot,
                    "type": mt5.ORDER_TYPE_BUY,
                    "price": price,
                    "sl": stloss,
                    "tp": tkprofit,
                    "deviation": deviation,
                    "magic": 234000,
                    "comment": "python script open",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_IOC,
                }
            elif position == 'SHORT':
                request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": lot,
                    "type": mt5.ORDER_TYPE_SELL,
                    "price": price,
                    "sl": stloss,
                    "tp": tkprofit,
                    "deviation": deviation,
                    "magic": 234000,
                    "comment": "python script open",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_IOC,
                }

            # send a trading request
            result = mt5.order_send(request)
            # check the execution result
            print("1. order_send(): by {} {} lots at {} with deviation={} points".format(symbol,lot,price,deviation));
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print("2. order_send failed, retcode={}".format(result.retcode))
                # request the result as a dictionary and display it element by element
                result_dict=result._asdict()
                for field in result_dict.keys():
                    print("   {}={}".format(field,result_dict[field]))
                    # if this is a trading request structure, display it element by element as well
                    if field=="request":
                        traderequest_dict=result_dict[field]._asdict()
                        for tradereq_filed in traderequest_dict:
                            print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
                print("shutdown() and quit")
                #mt5.shutdown()
                quit()


    print(purps)
    playsound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/done.mp3')

In [7]:
TIMER = []
iterator = 0

while iterator < 6:
    start = dt.now()
    brutus()
    end = dt.now()
    elapsed = end - start
    elapsed = elapsed.total_seconds()
    elapsed = round(elapsed/60,2)
    elapsed = str(elapsed)
    timer = elapsed + '   minutes'
    TIMER.append(timer)
    time.sleep(30)
    iterator = iterator + 1


print(TIMER)

1. order_send(): by CADJPY 0.5 lots at 77.259 with deviation=20 points
1. order_send(): by EURCHF 0.5 lots at 1.05976 with deviation=20 points
2. order_send failed, retcode=10019
   retcode=10019
   deal=0
   order=0
   volume=0.0
   price=0.0
   bid=0.0
   ask=0.0
   comment=No money
   request_id=0
   retcode_external=0
   request=TradeRequest(action=1, magic=234000, order=0, symbol='EURCHF', volume=0.5, price=1.05976, stoplimit=0.0, sl=1.06002, tp=1.059390805, deviation=20, type=1, type_filling=1, type_time=0, expiration=0, comment='python script open', position=0, position_by=0)
       traderequest: action=1
       traderequest: magic=234000
       traderequest: order=0
       traderequest: symbol=EURCHF
       traderequest: volume=0.5
       traderequest: price=1.05976
       traderequest: stoplimit=0.0
       traderequest: sl=1.06002
       traderequest: tp=1.059390805
       traderequest: deviation=20
       traderequest: type=1
       traderequest: type_filling=1
       tradere

In [ ]:
mt5.shutdown()